In [ ]:
import logging

logger = logging.getLogger("myllmet")
logger.setLevel(logging.DEBUG)
hdlr = logging.StreamHandler()
hdlr.setLevel(logging.NOTSET)
logger.addHandler(hdlr)

In [ ]:
import wikipediaapi
wiki_wiki = wikipediaapi.Wikipedia(user_agent="python-wikipediaapi", language="ja")

p_wiki = wiki_wiki.page("Claude")
context = p_wiki.text[:500]

print(context)

In [ ]:
import pandas as pd

df = pd.DataFrame([
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "Anthropicは元OpenAIのメンバーらが設立した企業で、倫理的なAI開発を重視している。\n"
            "Claudeは「フレンドリーで熱心な同僚」をコンセプトに設計されている。"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "AnthropicはGoogle傘下のAI研究部門で、対話型AIに特化している。\n"
            "Claudeは「フレンドリーで熱心な同僚」をコンセプトに設計されている。"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは2021年にMeta（旧Facebook）が開発し、2022年に全世界で同時公開された。\n"
            "リリース当初は「クラウディアAI」という名前で知られていた。\n"
            "このAIは主にSNS投稿の自動作成に使用されていた。"
        ),
        "context": context,
    },
    {
        "question": "Claudeの概要を説明してください",
        "answer": (
            "Claudeは、2023年3月14日にAnthropic社によって一般公開された対話型AIである。\n"
            "Anthropicは元OpenAIのメンバーらが設立した企業で、倫理的なAI開発を重視している。\n"
            "2025年現在、Claude4が最新版である。"
        ),
        "context": context,
    },
])
df

In [ ]:
from myllmet.io_aws import BedrockClient
from myllmet.metrics import Faithfulness
from myllmet.trackers import ListTracker

import boto3

bedrock_runtime_client = boto3.client("bedrock-runtime")

client = BedrockClient("anthropic.claude-3-haiku-20240307-v1:0")
tracker = ListTracker()
metrics = Faithfulness(client, client)

metrics.set_tracker(tracker)

In [ ]:
tmp = []
for r in df.itertuples(index=False):
    score = metrics.score(question=r.question, answer=r.answer, context=r.context)
    tmp.append(score)


df_score = df.copy()
df_score["score"] = tmp

df_score

In [ ]:
tracker.to_pandas("standard")

In [ ]:
df_int = tracker.to_pandas("intermediates")
df_int

In [ ]:
df_int.loc[1, "verdicts"]